In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image

import torch
import torchvision

import os

In [2]:
CSV_PATH = 'data.csv'

In [3]:
class IMD2020Dataset(torch.utils.data.Dataset):
    def __init__(self, csv_path):
        self.csv = pd.read_csv(csv_path)

        self.transforms_fake = torchvision.transforms.Compose([
            torchvision.transforms.ToTensor()
        ])

        self.transforms_mask = torchvision.transforms.Compose([
            torchvision.transforms.ToTensor()
        ])
    
    def __len__(self):
        return len(self.csv)

    def __getitem__(self, idx):
        row = self.csv.iloc[idx]
        fake, mask = row['fakes'], row['masks']

        fake = Image.open(fake)
        mask = Image.open(mask).convert('1')

        fake_t = self.transforms_fake(fake)
        mask_t = self.transforms_mask(mask)
        
        return fake_t, mask_t

In [4]:
ds = IMD2020Dataset(CSV_PATH)

In [5]:
len(ds)

2010

In [6]:
ds

In [7]:
for batch in ds:
    print(batch[0].size(), batch[1].size())

torch.Size([4, 503, 500]) torch.Size([1, 503, 500])
torch.Size([4, 555, 500]) torch.Size([1, 555, 500])
torch.Size([3, 654, 606]) torch.Size([1, 654, 606])
torch.Size([4, 474, 650]) torch.Size([1, 474, 650])
torch.Size([3, 474, 650]) torch.Size([1, 474, 650])
torch.Size([4, 474, 650]) torch.Size([1, 474, 650])
torch.Size([3, 474, 650]) torch.Size([1, 474, 650])
torch.Size([3, 633, 570]) torch.Size([1, 633, 570])
torch.Size([3, 633, 570]) torch.Size([1, 633, 570])
torch.Size([4, 550, 475]) torch.Size([1, 550, 475])
torch.Size([4, 699, 491]) torch.Size([1, 699, 491])
torch.Size([4, 624, 491]) torch.Size([1, 624, 491])
torch.Size([4, 699, 491]) torch.Size([1, 699, 491])
torch.Size([4, 566, 407]) torch.Size([1, 566, 407])
torch.Size([4, 528, 525]) torch.Size([1, 528, 525])
torch.Size([4, 528, 525]) torch.Size([1, 528, 525])
torch.Size([3, 639, 960]) torch.Size([1, 639, 960])
torch.Size([3, 368, 490]) torch.Size([1, 368, 490])
torch.Size([3, 611, 900]) torch.Size([1, 611, 900])
torch.Size([

KeyboardInterrupt: 